In [ ]:
## Import required libraries
import re
import pandas as pd
import requests
import urllib
import json
from time import sleep

In [ ]:
main_url = "https://api.undi.info/"
seat_types = ['state','parliament']
elections = ['ge11','ge12','ge13','ge14']

data_write_path = "data/raw/undi_dot_info/"

In [ ]:
def write_json(data_dict,filename,data_write_path=data_write_path):
    '''
    Write JSON file into designated raw data folder.
    '''
    
    # Serializing json
    json_object = json.dumps(data_dict, indent=4)

    # Writing to .json
    with open(data_write_path + filename, "w") as outfile:
        outfile.write(json_object)

In [ ]:
def read_json(filename,data_write_path=data_write_path):
    '''
    Read JSON file from designated raw data folder
    '''
    
    users = ''
    with open(data_write_path + filename, "r") as f:
        users = json.load(f)
        
    return users

In [ ]:
def get_undi_api(endpoint,params):
    '''
    Get data from undi.info API
    Params: endpoint and key value pairs of params
    Returns: JSON of payload
    '''
    
    url = main_url + endpoint + '?' + urllib.parse.urlencode(params)
    
    print("Requesting... "+ url)
    try:
        r = requests.get(url, timeout=5)
        print("Status: "+ str(r.status_code))
        return json.loads(r.text)
    except:
        print("Something went wrong")
    #sleep(2)
    
    return {}
    

## First we want to get all the codes for the States

In [ ]:
def get_state_codes():
    '''
    Use party_summary end point to get all the keys and parse state codes from there
    '''
    
    payload = get_undi_api('party_summary',{'ge': 'ge14', 'seat': 'state'})
    
    state_codes = [key[0:2].upper() for key in payload.keys()]
    
    return state_codes

In [ ]:
state_codes = get_state_codes()
state_codes

## Next we want to get all the Seat Names for both state and parliament level

In [ ]:
def get_seat_names(election,seat_type,state_codes = state_codes):
    '''
    Use listing end point to get all seat names with seat codes for a given seat type and election
    '''
    
    seats = []
    
    attributes = ['pcode','scode','name']
    
    for state in state_codes:
        payload = get_undi_api('listing',{'ge': election, 'seat': seat_type, 'negeri': state})
        if 'seats' in payload:
            for seat in payload['seats']:
                my_seat = {}
                for attribute in attributes:
                    if attribute in seat.keys():
                        my_seat[attribute] = seat[attribute]
                    my_seat['state'] = state
                seats.append(my_seat)
            
    return seats

In [ ]:
def get_all_seats_for_all_elections(elections = elections,seat_types = seat_types):
    '''
    Get all the seats for all the elections and for both parliament and state
    '''
    
    all_seats = {}
    
    for election in elections:
        all_seats[election] = {}
        for seat_type in seat_types:
            seats_for_this_election_and_type = get_seat_names(election,seat_type)
            all_seats[election][seat_type] = seats_for_this_election_and_type
            sleep(2)
            
    return all_seats

In [ ]:
#all_seats = get_all_seats_for_all_elections()

In [ ]:
#write_json(all_seats,"all_seats.json")

In [ ]:
all_seats = read_json('all_seats.json')

## Now we use the seat names data to build the params for the results API calls

In [ ]:
def build_result_seat_param(seat_type,state,seat_name):
    '''
    Build string for result seat param of the form [state code].[seat type code].[seat name]
    '''
    
    seat_type_code = 'p' if seat_type == 'parliament' else 'n'
    
    return state + '.' + seat_type_code + '.' + seat_name #.replace(' ','%20')

In [ ]:
def prep_results_params(all_seats):
    '''
    Prep all the URL params needed for getting election results API calls
    '''
    
    
    params = {}
    for seat_type in seat_types:
        params[seat_type] = []
        
        seats_for_seat_type = []
        for ge in all_seats:
            for seat in all_seats[ge][seat_type]:
                param = build_result_seat_param(seat_type,seat['state'],seat['name'])
                if param not in params[seat_type]:
                    params[seat_type].append(param)
                
    return params

In [ ]:
results_params = prep_results_params(all_seats)

In [ ]:
def get_results(results_params,seat_type):
    '''
    Make API calls to get all the seats in result_params for given seat_type
    '''
    
    results = []
    
    for param in results_params[seat_type]:
        print(param)
        [state,seat_type_code,seat_name] = [s.replace('%20',' ') for s in param.split(".")]

        results_from_api = get_undi_api('election',{'seat': param})

        if bool(results_from_api):
            results.append(results_from_api)
            
    return results

In [ ]:
results_parliament = get_results(results_params,'parliament')

In [ ]:
write_json(results_parliament,"results_parliament.json")

In [ ]:
results_states = get_results(results_params,'state')

In [ ]:
write_json(results_states,"results_states.json")